In [110]:
def get_classes_places365():
    import os
    file_name = 'categories_places365.txt'
    if not os.access(file_name, os.W_OK):
        synset_url = 'https://raw.githubusercontent.com/csailvision/places365/master/categories_places365.txt'
        os.system('wget ' + synset_url)
    classes = list()
    
    with open(file_name) as class_file:
        for line in class_file:
            classes.append(line.strip().split(' ')[0][3:])

    classes = tuple(classes)
    
    return classes

In [1]:
from src.models.keras_VGG16_places365.vgg16_places_365 import VGG16_Places365

Using TensorFlow backend.


In [133]:
import tensorflow as tf

In [2]:
from src.models import places_ontop_model
from src import custom_losses, custom_metrics, optimizers
from src.data import data

In [3]:
from src.models import base_model

In [4]:
from src.data import data

In [5]:
paths = data.PATH()
dataset_path = f'{paths.PROCESSED_DATA_PATH}/'
dataset = 'vision_based_for_places_cnn'
test_dataset_path = f'{dataset_path}/{dataset}/'

In [6]:
batch_size = 64
n_classes = 365
epochs = 100
img_size = 224
n_channels = 3

In [7]:
model = base_model.BaseModel(batch_size, n_classes, epochs, img_size, n_channels, experiment_name=f'exp_3')

In [8]:
model.model = VGG16_Places365(weights='places')

W0105 12:09:34.156316 139797413492544 deprecation_wrapper.py:119] From /home/ifranco/Documents/facultad/tesis/tesis_env/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [9]:
validation_generator, test_generator = model.get_image_data_generator(test_dataset_path, train=False, validation=True, test=True, class_mode_validation=None, class_mode_test=None)

Found 6305 images belonging to 1 classes.
Found 6328 images belonging to 1 classes.


In [10]:
validation_generator.class_indices.keys()
validation_generator.classes

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [11]:
model.model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [13]:
pred_proba_validation = model.predict_from_generator(test_dataset_path, test_generator=validation_generator, return_pred_proba=True)

Found 6305 images belonging to 1 classes.


In [14]:
pred_proba_test = model.predict_from_generator(test_dataset_path, test_generator=test_generator, return_pred_proba=True)

Found 6305 images belonging to 1 classes.


In [90]:
import numpy as np

In [91]:
def get_label_prediction(pred_proba):
    label_prediction = np.argmax(pred_proba, axis=1)
    return label_prediction

In [92]:
def get_prediction_confidence(pred_proba):
    confidence = np.max(pred_proba, axis=1)
    return confidence

In [93]:
def get_y_true_from_generator(generator, return_filename=False):
    y_true = [f.split('/')[1] for f in generator.filenames]
    
    if return_filename:
        filenames = [f.split('/')[2] for f in generator.filenames]
        return y_true, filenames
    return y_true

In [94]:
def get_pred_label_and_confidence(pred_proba):
    classes = get_classes_places365()
    label_prediction, pred_confidence = (
        get_label_prediction(pred_proba),
        get_prediction_confidence(pred_proba))
    
    return [(classes[l], pred_confidence[idx]) for idx, l in enumerate(label_prediction)]

In [1]:
import pandas as pd

In [96]:
def get_df_pred(model, generator, dataset_path):
    pred_proba = model.predict_from_generator(dataset_path, 
                                              test_generator=generator, 
                                              return_pred_proba=True)
    
    y_pred = get_pred_label_and_confidence(pred_proba)
    y_true, filenames = get_y_true_from_generator(generator, 
                                                  return_filename=True)
    return pd.DataFrame({
        'y_true': y_true,
        'y_pred': [i[0] for i in y_pred],
        'y_pred_proba': [i[1] for i in y_pred],
        'filename': filenames
        })

In [97]:
df_pred_test = get_df_pred(model, test_generator, test_dataset_path)

Found 6305 images belonging to 1 classes.


In [98]:
df_pred_val = get_df_pred(model, validation_generator, test_dataset_path)

Found 6305 images belonging to 1 classes.


In [68]:
classes = get_classes_places365()

In [71]:
# pd.DataFrame(list(classes)).to_csv('mapping_places365.csv')

In [2]:
df_mapping = pd.read_csv('mapping_places365.csv', )

In [3]:
df_mapping = df_mapping[['class', 'mapped']]

In [4]:
df_mapping.fillna('other', inplace=True)

In [116]:
for _, row in df_mapping.iterrows():
    c, m = row['class'], row['mapped']
    
    mask = df_pred_test['y_pred'] == c
    df_pred_test.loc[mask, 'y_pred_mapped'] = m
    mask = df_pred_val['y_pred'] == c
    df_pred_val.loc[mask, 'y_pred_mapped'] = m

In [121]:
numerical_classes = {c: idx for idx, c in enumerate(df_pred_test.y_true.unique())}

In [123]:
numerical_classes['other'] = 6

In [129]:
for c, idx in numerical_classes.items():
    mask = df_pred_test['y_true'] == c
    df_pred_test.loc[mask, 'y_true_num'] = idx
    
    mask = df_pred_test['y_pred_mapped'] == c
    df_pred_test.loc[mask, 'y_pred_num'] = idx
    
    mask = df_pred_val['y_true'] == c
    df_pred_val.loc[mask, 'y_true_num'] = idx
    
    mask = df_pred_val['y_pred_mapped'] == c
    df_pred_val.loc[mask, 'y_pred_num'] = idx

In [180]:
import tensorflow as tf

In [185]:
cat_acc = tf.keras.metrics.categorical_accuracy

In [208]:
from sklearn.metrics import accuracy_score

In [ ]:
y_true = df_pred['y_true_num']
y_pred = df_pred['y_pred_num']

In [ ]:
model.model.evaluate(y_true)

In [234]:
def get_categorical_accuracy_for_pred(df_pred):
    cases = df_pred.loc[df_pred['y_pred_num'] == df_pred['y_true_num']]
    y_true = df_pred['y_true_num']
    y_pred = df_pred['y_pred_num']
    #return accuracy_score(y_true, y_pred), cases
    acc = accuracy_score(y_true, y_pred)
    
    y_true = tf.keras.utils.to_categorical(
        df_pred['y_true_num'],
        num_classes=7,
        dtype='float32'
    )
    y_pred = tf.keras.utils.to_categorical(
        df_pred['y_pred_num'],
        num_classes=7,
        dtype='float32'
    )
    m = cat_acc(y_true, y_pred)

    with tf.Session() as sess:
        t = sess.run(m)
        
    return t.sum() / t.shape[0], acc, cases

In [237]:
result_test = get_categorical_accuracy_for_pred(df_pred_test)

In [240]:
result_test[:2]

(0.05625790139064475, 0.05625790139064475)

In [238]:
result_val = get_categorical_accuracy_for_pred(df_pred_val)

In [241]:
result_val[:2]

(0.049643140364789846, 0.049643140364789846)

In [17]:
df_mapping['class2'] = df_mapping['class'].shift(-35)
df_mapping['mapped2'] = df_mapping['mapped'].shift(-35)

In [19]:
df_mapping[:35]

,class,mapped,class2,mapped2
0,airfield,other,ballroom,other
1,airplane_cabin,other,bamboo_forest,other
2,airport_terminal,other,bank_vault,other
3,alcove,other,banquet_hall,other
4,alley,other,bar,other
5,amphitheater,other,barn,other
6,amusement_arcade,other,barndoor,other
7,amusement_park,other,baseball_field,other
8,apartment_building/outdoor,frontyard,basement,other
9,aquarium,other,basketball_court/indoor,other


In [23]:
print(df_mapping.to_latex(index=False))

\begin{tabular}{llll}
\toprule
                      class &       mapped &                    class2 &      mapped2 \\
\midrule
                   airfield &        other &                  ballroom &        other \\
             airplane\_cabin &        other &             bamboo\_forest &        other \\
           airport\_terminal &        other &                bank\_vault &        other \\
                     alcove &        other &              banquet\_hall &        other \\
                      alley &        other &                       bar &        other \\
               amphitheater &        other &                      barn &        other \\
           amusement\_arcade &        other &                  barndoor &        other \\
             amusement\_park &        other &            baseball\_field &        other \\
 apartment\_building/outdoor &    frontyard &                  basement &        other \\
                   aquarium &        other &   basketball\_co

In [21]:
df_mapping[30:40]

,class,mapped,class2,mapped2
30,badlands,other,boxing_ring,other
31,bakery/shop,other,bridge,other
32,balcony/exterior,other,building_facade,frontyard
33,balcony/interior,other,bullring,other
34,ball_pit,other,burial_chamber,other
35,ballroom,other,bus_interior,other
36,bamboo_forest,other,bus_station/indoor,other
37,bank_vault,other,butchers_shop,other
38,banquet_hall,other,butte,other
39,bar,other,cabin/outdoor,other


In [11]:
print(df_mapping.to_latex(index=False))

\begin{tabular}{ll}
\toprule
                      class &       mapped \\
\midrule
                   airfield &        other \\
             airplane\_cabin &        other \\
           airport\_terminal &        other \\
                     alcove &        other \\
                      alley &        other \\
               amphitheater &        other \\
           amusement\_arcade &        other \\
             amusement\_park &        other \\
 apartment\_building/outdoor &    frontyard \\
                   aquarium &        other \\
                   aqueduct &        other \\
                     arcade &        other \\
                       arch &        other \\
   archaelogical\_excavation &        other \\
                    archive &        other \\
               arena/hockey &        other \\
          arena/performance &        other \\
                arena/rodeo &        other \\
                  army\_base &        other \\
                art\_gallery &     

In [90]:
import numpy as np

def get_label_prediction(pred_proba):
    label_prediction = np.argmax(pred_proba, axis=1)
    return label_prediction

def get_prediction_confidence(pred_proba):
    confidence = np.max(pred_proba, axis=1)
    return confidence

def get_y_true_from_generator(generator, return_filename=False):
    y_true = [f.split('/')[1] for f in generator.filenames]
    
    if return_filename:
        filenames = [f.split('/')[2] for f in generator.filenames]
        return y_true, filenames
    return y_true

def get_pred_label_and_confidence(pred_proba):
    classes = get_classes_places365()
    label_prediction, pred_confidence = (
        get_label_prediction(pred_proba),
        get_prediction_confidence(pred_proba))
    
    return [(classes[l], pred_confidence[idx]) for idx, l in enumerate(label_prediction)]

import pandas as pd

def get_df_pred(model, generator, dataset_path):
    pred_proba = model.predict_from_generator(dataset_path, 
                                              test_generator=generator, 
                                              return_pred_proba=True)
    
    y_pred = get_pred_label_and_confidence(pred_proba)
    y_true, filenames = get_y_true_from_generator(generator, 
                                                  return_filename=True)
    return pd.DataFrame({
        'y_true': y_true,
        'y_pred': [i[0] for i in y_pred],
        'y_pred_proba': [i[1] for i in y_pred],
        'filename': filenames
        })

df_pred_test = get_df_pred(model, test_generator, test_dataset_path)

df_pred_val = get_df_pred(model, validation_generator, test_dataset_path)